In [1]:
from gurobipy import *

This is the optimization for a rack by rack basis in the system. 

In [2]:
m = Model();

Academic license - for non-commercial use only


The decision varible is $x_{ijr}$ which is the number of chips where $i$ is the specific chip type, the $j$ is the orientation, and $r$ is the rack where the chip is assigned.

In [3]:
chips, powerdraw, quantity = multidict({
    '1080':[150,10],
    '2080':[180,10]
})

In [4]:
racks , slots = multidict({
    'alpha':48,
    'beta':48
})

The orientations are as follows: 
$$j_1\rightarrow(L1,L2)$$
$$j_2\rightarrow(L2,L3)$$
$$j_3\rightarrow(L1,L3)$$
$$s.t.$$
$$\theta = \{j_1,j_2,j_3\}$$

In [5]:
i = ['1080','2080']

In [6]:
j = [0,1,2]

In [7]:
r = ['alpha']

In [8]:
x = m.addVars(i,j,r,vtype=GRB.INTEGER)

In [9]:
x

{('1080', 0, 'alpha'): <gurobi.Var C0>,
 ('1080', 1, 'alpha'): <gurobi.Var C1>,
 ('1080', 2, 'alpha'): <gurobi.Var C2>,
 ('2080', 0, 'alpha'): <gurobi.Var C3>,
 ('2080', 1, 'alpha'): <gurobi.Var C4>,
 ('2080', 2, 'alpha'): <gurobi.Var C5>}

In [10]:
# make sure all the chips have somewhere to go
m.addConstrs(x.sum(chip,'*','*') == quantity[chip] for chip in chips)

{'1080': <gurobi.Constr *Awaiting Model Update*>,
 '2080': <gurobi.Constr *Awaiting Model Update*>}

In [11]:
L = m.addVars(racks,range(3))

In [12]:
L

{('alpha', 0): <gurobi.Var *Awaiting Model Update*>,
 ('alpha', 1): <gurobi.Var *Awaiting Model Update*>,
 ('alpha', 2): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 0): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 1): <gurobi.Var *Awaiting Model Update*>,
 ('beta', 2): <gurobi.Var *Awaiting Model Update*>}

In [13]:
# we want to perfer l1-l2, l2-l3, l1-l3 in that order
# easier to work with the top of the rack
m.addConstrs(L[rack,0]>=L[rack,1] for rack in racks)
m.addConstrs(L[rack,1]>=L[rack,2] for rack in racks)

{'alpha': <gurobi.Constr *Awaiting Model Update*>,
 'beta': <gurobi.Constr *Awaiting Model Update*>}

In [14]:
# the max diff in a rack
w = m.addVars(r) 

In [16]:
# calc the actual diff in a rack
m.addConstr(L['alpha',2]-L['alpha',0] == w['alpha'])

<gurobi.Constr *Awaiting Model Update*>

In [21]:
m.addConstr(L['alpha',2]>=L['alpha',1])
m.addConstr(L['alpha',1]>=L['alpha',0])

<gurobi.Constr *Awaiting Model Update*>

In [22]:
m.setObjective(quicksum(w[r))

NameError: name 'rack' is not defined

In [19]:
m.addConstr(quicksum(powerdraw[chip]*x[chip,0,'alpha'] for chip in chips) + quicksum(powerdraw[chip]*x[chip,2,'alpha'] for chip in chips)  == L['alpha',0])
m.addConstr(quicksum(powerdraw[chip]*x[chip,0,'alpha'] for chip in chips) + quicksum(powerdraw[chip]*x[chip,1,'alpha'] for chip in chips)  == L['alpha',1])
m.addConstr(quicksum(powerdraw[chip]*x[chip,1,'alpha'] for chip in chips) + quicksum(powerdraw[chip]*x[chip,2,'alpha'] for chip in chips)  == L['alpha',2])

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
m.optimize()

In [ ]:
x